In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (C:\Users\RICHARDSON\Anaconda3\lib\site-packages\PIL\__init__.py)

In [5]:
# USe Cuda GPU 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
torch.cuda.is_available()

False

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(), # CHange it to Tensor 
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module): #CNN module inheritance 

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 1, 28, 28) (batch size, input channel, height, width)
        #    Conv     -> (?, 32, 28, 28) output size = {28 (input - filter ) + 2*padding}/Stride
        #    Pool     -> (?, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), #inputchaneel=1,outputchannet =32,kernal =3*3
            torch.nn.ReLU(), #activation fuction
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) #Maxpooling kernel size 2*2
        
        
        # L2 ImgIn shape=(?, 32, 14, 14)
        #    Conv      ->(?, 64, 14, 14) output size = 14
        #    Pool      ->(?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), #input =32, output=64, kernel = 3*3
            torch.nn.ReLU(), #activation fuction
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True) #7*7*64 information linera, output (0-9)
        torch.nn.init.xavier_uniform_(self.fc.weight) #torch paramter initialization 

    def forward(self, x):
        out = self.layer1(x) # Passing through first layer 
        out = self.layer2(out) #Passing through second layer
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [7]:
# instantiate CNN model
model = CNN().to(device)

In [8]:
# define cost function Cross Entorypy function 
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.

#Adam optimizaer set up 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# train my model
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device) # Image data
        Y = Y.to(device) # Label data

        hypothesis = model(X) # Hypothersis
        cost = criterion(hypothesis, Y) #Cost calcualtion 
        
        optimizer.zero_grad() # Initialization Gradeint >>0 
        cost.backward() #Gradeint calculation 
        optimizer.step() # update

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 0.223760188
[Epoch:    2] cost = 0.0620357655
[Epoch:    3] cost = 0.0448702686
[Epoch:    4] cost = 0.0355475172
[Epoch:    5] cost = 0.0290100053
[Epoch:    6] cost = 0.0249212869
[Epoch:    7] cost = 0.0207808968
[Epoch:    8] cost = 0.0181876235
[Epoch:    9] cost = 0.0150148766
[Epoch:   10] cost = 0.0125987045
[Epoch:   11] cost = 0.0104575502
[Epoch:   12] cost = 0.00984687544
[Epoch:   13] cost = 0.0080845654
[Epoch:   14] cost = 0.00733984308
[Epoch:   15] cost = 0.00562915765
Learning Finished!


In [15]:
# Test model and check accuracy
import matplotlib.pyplot as plt
import random
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())    


Accuracy: 0.986299991607666
